### Install the required packages

In [ ]:
# %pip install python-dotenv
# %pip install requests
# %pip install azure-storage-blob
# %pip install azure-storage-file-datalake

### Load the configuration settings

In [ ]:
from dotenv import dotenv_values,load_dotenv

import sys
sys.path.append('../common')
from helper import Helper


query_params = "?api-version=2024-05-01"
# Load the environment variables
load_dotenv()

secrets = dotenv_values("secrets.env")
api_key = secrets.get("API_KEY")
environment = secrets.get("APIM_NAME")
region_override = secrets.get("REGION_OVERRIDE")

helper = Helper(environment, "translator/azure/document", api_key, region_override)

### Get supported documents

In [ ]:
response = helper.get("/translator/document/formats" + query_params + "&type=document")

assert response.status_code == 200

response_json = response.json
if 'value' in response_json and response_json['value']:
    assert len(response_json['value']) > 0
else:
    raise ValueError("The response does not contain 'value' property or it has no children.")

### Prepare storage

In [ ]:
from azure.storage.blob import BlobServiceClient, generate_container_sas, ContainerSasPermissions
from azure.storage.filedatalake import DataLakeServiceClient
from datetime import datetime, timedelta, timezone

account_connection_string = secrets.get("AZURE_STORAGE_CONNECTION_STRING")
source_container_name = "translator-source"
destination_container_name = "translator-destination"

blob_service_client = BlobServiceClient.from_connection_string(account_connection_string)
data_lake_client = DataLakeServiceClient.from_connection_string(account_connection_string)

account_name = blob_service_client.account_name

### Generate SAS tokens

In [ ]:
source_sas_token = generate_container_sas(
    account_name=account_name,
    container_name=source_container_name,
    account_key=blob_service_client.credential.account_key,
    permission=ContainerSasPermissions(read=True, list=True),
    expiry=datetime.now(timezone.utc) + timedelta(hours=1)
)

destination_sas_token = generate_container_sas(
    account_name=account_name,
    container_name=destination_container_name,
    account_key=blob_service_client.credential.account_key,
    permission=ContainerSasPermissions(read=True, list=True, write=True, delete=True, add=True, create=True),
    expiry=datetime.now(timezone.utc) + timedelta(hours=1)
)

source_sas_url = f"https://{account_name}.blob.core.windows.net/{source_container_name}?{source_sas_token}"
destination_sas_url = f"https://{account_name}.blob.core.windows.net/{destination_container_name}?{destination_sas_token}"

helper.log(f"Source SAS URL: {source_sas_url}")
helper.log(f"Destination SAS URL: {destination_sas_url}")

### Check if source container contains files

In [ ]:
from azure.storage.blob import ContainerClient

# Create a ContainerClient using the source SAS URL
source_container_client = ContainerClient.from_container_url(source_sas_url)

# List all blobs in the container
source_blobs = list(source_container_client.list_blobs())

# Assert that there are blobs in the container
assert len(source_blobs) > 0, "No blobs found in the source container."

print(f"Found {len(source_blobs)} blobs in the source container.")

### Delete any files from destination container

In [ ]:
from azure.storage.blob import ContainerClient

# Create a ContainerClient using the destination SAS URL
container_client = ContainerClient.from_container_url(destination_sas_url)

# List all blobs in the container
blobs = container_client.list_blobs()

# Delete each blob
for blob in blobs:
    blob_client = container_client.get_blob_client(blob)
    blob_client.delete_blob()
    print(f"Deleted blob: {blob.name}")

print("All files in the destination directory have been deleted.")

### Start batch translation

In [ ]:
body = {
 "inputs": [
   {
     "source": {
       "sourceUrl": source_sas_url,
     },
     "targets": [
       {
         "targetUrl": destination_sas_url,
         "language": "fr"
       }
     ]
   }
 ]
}

helper.log(f"Request body: {body}")

response = helper.post("/translator/document/batches" + query_params , body)

assert response.status_code == 202

response_json = response.json
assert len(response_json) > 0

assert 'id' in response_json, "Response does not contain 'id' property"
assert 'status' in response_json, "Response does not contain 'status' property"

job_id = response_json['id']


### Get status for a specific job

In [ ]:
import time

done = False

while(done != True):
    response = helper.get("/translator/document/batches/"+ job_id + "/" + query_params + "&type=document")

    assert response.status_code == 200

    response_json = response.json
    assert len(response_json) > 0

    assert 'id' in response_json, "Response does not contain 'id' property"
    assert 'status' in response_json, "Response does not contain 'status' property"
    status = response_json['status']
    done = not status in ['NotStarted', 'Running']
    time.sleep(5)

assert status == 'Succeeded', f"Job did not succeed, current status: {status}"


### Get status for all documents

In [ ]:
response = helper.get("/translator/document/batches/"+ job_id + "/documents" + query_params)

assert response.status_code == 200

response_json = response.json
if 'value' in response_json and response_json['value']:
    assert len(response_json['value']) > 0
else:
    raise ValueError("The response does not contain 'value' property or it has no children.")

first_file = response_json['value'][0]

assert 'path' in first_file, "Response does not contain 'path' property"
assert 'status' in first_file, "Response does not contain 'status' property"
assert 'id' in first_file, "Response does not contain 'id' property"

document_id = first_file['id']


### Get status for a specific document

In [ ]:
response = helper.get("/translator/document/batches/"+ job_id + "/documents/"+ document_id + query_params)

assert response.status_code == 200

response_json = response.json
assert len(response_json) > 0

assert 'path' in response_json, "Response does not contain 'path' property"
assert 'status' in response_json, "Response does not contain 'status' property"

### Cancel translation

In [ ]:
response = helper.delete("/translator/document/batches/" + job_id + query_params)

assert response.status_code == 200

response_json = response.json
assert len(response_json) > 0

assert 'id' in response_json, "Response does not contain 'id' property"
assert 'status' in response_json, "Response does not contain 'status' property"